We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

In [34]:
import pandas as pd

df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
len(df.columns)

19

In [35]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


Q1: How many columns are there?
 - 19

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [36]:
df.loc[:, 'duration'] = df.loc[:, 'tpep_dropoff_datetime'] - df.loc[:, 'tpep_pickup_datetime']
df.loc[:, 'duration'] = df.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/4186165970.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[17.81666667  8.4         8.96666667 ... 11.         12.05
 27.        ]' has dtype incompatible with timedelta64[us], please explicitly cast to a compatible dtype first.
  df.loc[:, 'duration'] = df.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)


In [37]:
df.duration.std()

46.44530513776802

Q2: What's the standard deviation of the trips duration in January?
- 46.45

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [38]:
# before
print(len(df))
df = df[(df.duration >= 1) & (df.duration <= 60)]
print(len(df))

2463931
2421440


In [39]:
2421440/2463931

0.9827547930522406

Q3: What fraction of the records left after you dropped the outliers?
- 98%

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

In [41]:
categorical = ['PULocationID', 'DOLocationID']
df.loc[:, categorical] = df.loc[:,categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

In [43]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [46]:
X_train.shape

(2421440, 515)

Q4: What's the dimensionality of this matrix (number of columns)?
- 515

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.9861908032968065

Q5: What's the RMSE on train?
- 6.99

Now let's apply this model to the validation dataset (February 2022).

In [48]:
val = pd.read_parquet('yellow_tripdata_2022-02.parquet')
val.loc[:, 'duration'] = val.loc[:, 'tpep_dropoff_datetime'] - val.loc[:, 'tpep_pickup_datetime']
val.loc[:, 'duration'] = val.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)
val = val[(val.duration >= 1) & (val.duration <= 60)]
val.loc[:, categorical] = val.loc[:,categorical].astype(str)
val_dicts = val[categorical].to_dict(orient='records')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/184767930.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[12.43333333 17.55       23.65       ... 15.33333333  8.26666667
  6.        ]' has dtype incompatible with timedelta64[us], please explicitly cast to a compatible dtype first.
  val.loc[:, 'duration'] = val.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/184767930.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['138' '138' '138' ... '161' '141' '237']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  val.loc[:, categorical] = val.loc[:,categorical].astype(str)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/184767930.py:5: FutureWarning: Setting an item of incompatible dtype is dep

In [49]:
X_val = dv.transform(val_dicts)
y_train = df[target].values
y_val = val[target].values

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.786407783246607

Q6: What's the RMSE on validation?
- 7.79